# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [4]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [5]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [6]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [7]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [8]:
# Connect to a Cassandra instance in local machine 

from cassandra.cluster import Cluster
cluster = Cluster()


# Connect to a session
session = cluster.connect()

#### Create Keyspace

In [9]:
# Create a keyspace to store the tables 
try: 
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =  {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")

except Exception as e: 
    print(e)
    

#### Set Keyspace

In [10]:
# Set the keyspace to udacity
try: 
    session.set_keyspace('udacity')
except Exception as e:
        print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1 - Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### Create Table music_library

In [11]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
# First we start with creating a music_library table  with the columns: session_id, item_in_session, artist_name, song_title, and song duration.
# To ensure that fields are unique, the PRIMARY KEY willbe the following:  ((sessionId), item_in_session) and PARTITION KEY to be sessionId
# itemInSession will serve as a clustering column and is added in the PRIMARY KEY 


#CREATE TABLE music_library
query = "CREATE TABLE IF NOT EXISTS music_library "
query = query + "(session_id INT, item_in_session INT, artist_name VARCHAR, song_title text, song_duration DECIMAL, PRIMARY KEY ((session_id), item_in_session))"

try:
    session.execute(query)
except Exception as exep:
    print(exep)
                    

#### Load CSV file into table music_library

In [12]:
# Load the data from the CSV file and insert into the Cassandra table music_library
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
 # Insert data into the table
        query = "INSERT INTO music_library (session_id, item_in_session, artist_name, song_title, song_duration) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## Properly assign and direct the values from CSV to the table attributes
        ## Example of an assignment would be:# SELECT statement to verify whether the data was properly entered into the table first_name, you would change the code below to `line[0], line[1]`
        
        
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as exep:
            print(exep)

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
# Utilize a SELECT statement query to verify whether the data was properly entered into the table

select_query = "SELECT artist_name, song_title, song_duration FROM music_library WHERE session_id=338 AND item_in_session=4"

try:
    rows = session.execute(select_query)
except Exception as exep:
    print(exep)

# Print the results
for row in rows:
    print(row.artist_name, row.song_title, row.song_duration)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Query 2 - Provide the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create table artist_library

In [14]:
## Query 2 Provide only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
#  First we start with creating a table called artist_library with the following columns: user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name. 
#  To ensure the fields are the PRIMARY KEY will be the following: ((user_id, sessionId), item_in_session) and PARTITION KEY to be (user_id, sessionId)


#CREATE TABLE artist_library

query = "CREATE TABLE IF NOT EXISTS artist_library "
query = query + "(user_id INT, session_id INT, item_in_session INT, artist_name VARCHAR, song_title TEXT, first_name VARCHAR, last_name VARCHAR, \
                    PRIMARY KEY ((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as exep:
    print(exep)
                    

#### Load CSV file and insert data into artist_library

In [15]:
# Load the data from the CSV file and insert into the Cassandra table artist_library
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        
        #  Insert data into the table artist_library
        
        query = "INSERT INTO artist_library (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Properly assign and direct values from CSV to the table attributes
        
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as exep:
            print(exep)
                    

#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
# Utilize a SELECT statement query to verify whether the data was properly entered into the table
select_query = "SELECT artist_name, song_title, first_name, last_name FROM artist_library WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(select_query)
except Exception as exep:
    print(exep)

# Print the results
for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3 - Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own

#### Create table play_history

In [18]:
## Query 3: Provide every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# First we must create a table called play_history with the following fields: song_title, user_id, first_name, last_name
# The ensure that each record is unique, PRIMARY KEY will be the following: ((song_title), userId) and PARTITION KEY to be song_title
 

#CREATE TABLE play_history

query = "CREATE TABLE IF NOT EXISTS play_history "
query = query + "(song_title TEXT, user_id INT, first_name VARCHAR, last_name VARCHAR, PRIMARY KEY ((song_title), user_id))"

try:
    session.execute(query)
except Exception as exep:
    print(exep)

                    

#### Load CSV file and insert data into play_history

In [19]:
# Load the data from the CSV file and insert into the Cassandra table play_history

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        # Insert into the table play_history
        
        query = "INSERT INTO play_history (song_title, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        
        # Properly assign the values from CSV to the table attributes
        
        try:
            session.execute(query, (line[1], int(line[10]), line[4], line[9]))
        except Exception as exep:
            print(exep)

#### Do a SELECT to verify that the data have been inserted into each table

In [20]:
# Utilize a SELECT statement query to verify whether the data was properly entered into the table

select_query = "SELECT first_name, last_name FROM play_history WHERE song_title='All Hands Against His Own'"

try:
    rows = session.execute(select_query)
except Exception as exep:
    print(exep)

# Print the results
for row in rows:
    print(row.artist_name, row.song_title, row.song_duration)

### Drop the tables before closing out the sessions

In [21]:
## TO-DO: Drop the table before closing out the sessions

try:
    session.execute("DROP TABLE IF EXISTS music_library")
except Execption as exep:
    print(exep)
    
try:
    session.execute("DROP TABLE IF EXISTS artist_library")
except Execption as exep:
    print(exep)
    
try:
    session.execute("DROP TABLE IF EXISTS play_history")
except Execption as exep:
    print(exep)  

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()